In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('버스정류장.csv')

In [5]:
df.head()

,정류소 명,정류소 번호,정류소아이디,행정동 명,위도,경도
0,(구)주안2동치안센터,37302.0,163000302,주안2동,37.457078,126.671236
1,(주)경동세라믹스,89146.0,168001146,오류왕길동,37.559186,126.616978
2,(주)경인양행앞,42096.0,168000096,가좌1동,37.499045,126.665414
3,(주)경인양행앞,42097.0,168000097,석남2동,37.499342,126.666789
4,(주)대한특수금속,39050.0,165000050,논현고잔동,37.400029,126.694491


In [7]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# 시설 위도, 경도 배열
locations = df[['위도', '경도']].to_numpy()

# 위도, 경도를 라디안으로 변환
locations_rad = np.radians(locations)

# k-NN 모델 초기화 (3개 최근접 시설 찾기)
nbrs = NearestNeighbors(n_neighbors=3, metric='haversine')
nbrs.fit(locations_rad)

# 사용자 위치 (예: 위도, 경도)
user_location = (37.542996, 126.677156)  # 학원 위치
user_loc_rad = np.radians([user_location])

# 최근접 3개 시설 찾기
distances, indices = nbrs.kneighbors(user_loc_rad)

# 거리(km)로 변환 (각 시설마다)
distances_km = distances[0] * 6371  

# 찾은 3개 시설 정보 출력
for i, (idx, dist) in enumerate(zip(indices[0], distances_km), start=1):
    facility = df.iloc[idx]
    print(f"{i}번째 가까운 시설: {facility['정류소 명']}")
    print(f"정류소아이디: {facility['정류소아이디']}")
    print(f"거리: {dist:.2f} km\n")


1번째 가까운 시설: 서구청역(3번출구)
정류소아이디: 168000284
거리: 0.05 km

2번째 가까운 시설: 서부경찰서
정류소아이디: 168000286
거리: 0.11 km

3번째 가까운 시설: 서인천농협
정류소아이디: 168000277
거리: 0.16 km



In [8]:
import folium

# 인천 중심 좌표 설정
incheon_center = [37.456255, 126.705206]

# 기본 지도 생성
m = folium.Map(location=incheon_center, zoom_start=11)

In [9]:
m

In [10]:
# 사용자 위치 마커 추가
folium.Marker(location=user_location, popup='내 위치', icon=folium.Icon(color='red')).add_to(m)

# 가장 가까운 3개 시설 마커 강조 표시 (색상 변경 등)
for idx in indices[0]:
    facility = df.iloc[idx]
    folium.Marker(location=[facility['위도'], facility['경도']],
                  popup=facility['정류소 명'],
                  icon=folium.Icon(color='green')).add_to(m)
    


In [11]:
m